## Building Classification Models and Parameter Tunning?

#Brief Explanation 

### Load Data and Compute TFIDF Matrix

In [1]:
def scikit_TFIDF(n_feat,Train,Test):
    
    '''This method returns the TFIDF matrixes for both training and testing data'''
    t=TfidfVectorizer(max_features=n_feat)
    tfidf_train = t.fit_transform(Train.values.astype('U')).toarray()
    tfidf_test= t.transform(Test.values.astype('U')).toarray()
    
    return tfidf_train,tfidf_test

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from textblob import TextBlob

# Load the data
positive=pd.read_table("positive-words.txt")
negative=pd.read_table("negative-words.txt")

#Load Sentiment104 Processed Data 
Sentiment=pd.read_csv("Tweets_50kProc.csv")
y=Sentiment.label
X=Sentiment.tweets


#Make sure that the tweets have correct language



#40,000 and #10,000
train_X,test_X,train_y,text_y=train_test_split(X,y,test_size=0.2,random_state=123)

#Compyte the TFIDF matrix
tfidf_Train,tfidf_Test=scikit_TFIDF(20000,train_X,test_X)

TFIDF Matrix is computed as... 

### Baseline Model 

This model just looks at count data and see if words in the text are in the positive or negative list and makes classification accordingly.


In [72]:
#from textblob import TextBlob
#ind = [TextBlob(text).word_counts for text in X)]

### SVM Using TFIDF with hinge loss

In [3]:
import pandas as pd
from sklearn import svm
import csv
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Build SVM Model
model = svm.LinearSVC(max_iter = 10000, intercept_scaling = 1, C=1,loss = 'hinge')
# Fit Model
model.fit(tfidf_Train, train_y)
# Prediction
pred = model.predict(tfidf_Test)



In [4]:
print(classification_report(pred,text_y))

             precision    recall  f1-score   support

          0       0.78      0.77      0.78      5022
          4       0.77      0.78      0.78      4978

avg / total       0.78      0.78      0.78     10000



In [5]:
print(accuracy_score(pred,text_y))

0.7759


In [6]:
import numpy as np

# Build SVM Model
model = svm.LinearSVC(max_iter = 10000, intercept_scaling = 1, C=10,loss = 'hinge')
# Fit Model
model.fit(tfidf_Train, train_y)
# Prediction
pred = model.predict(tfidf_Test)
#Save accuracy
print(accuracy_score(pred,text_y))

0.7537


In [ ]:
# Build SVM Model
model = svm.LinearSVC(max_iter = 10000, intercept_scaling = 1, C=0.1,loss = 'hinge')
# Fit Model
model.fit(tfidf_Train, train_y)
# Prediction
pred = model.predict(tfidf_Test)
#Save accuracy
print(accuracy_score(pred,text_y))

0.7637


Since the value of C controls the margins for the SVM classifier, I want to test 3 different values of C to see if there is an effect in the accuracy.


### What about using other kernels? Gaussian and Sigmoid

In [ ]:
rbf_SVM=svm.SVC(kernel="rbf",C=1,gamma=5)
rbf_SVM.fit(tfidf_Train, train_y)
pred=rbf_SVM.predict(tfidf_Test)
print(classification_report(pred,text_y))

### Neural Network with TFDIF

### Sentiment Classifier https://pypi.org/project/sentiment_classifier/
    

In [91]:
from senti_classifier import senti_classifier
pos_score, neg_score = senti_classifier.polarity_scores()



1

## Performance of Models

In [ ]:
CONFUSION MATRICES, AUC ROC CURVES